In [2]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))

In [ ]:
import ee 
import geemap

# ee.Initialize(project="ee-india-reservoirs")
# ee.Initialize(project="ee-sahellakes")
ee.Initialize(project="thurgau-irrigation")


In [4]:
from src.et_green.compute_et_green import compute_et_green, calculate_band_std_dev
from src.et_green.filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
    get_unique_nutzung,
)
from src.et_green.exporting_utils import (
    process_et_green, 
    prepare_rainfed_fields, 
    process_et_green_rainfed_buffered
)

from utils.ee_utils import (
    back_to_float,
    back_to_int,
    export_image_to_asset,
    print_value_ranges,
    is_image_empty,
    fill_gaps_with_zeros
)

from utils.date_utils import print_collection_dates, merge_same_date_images

---

## Constants

In [28]:
YEAR = 2023

# KANTON = "FribourgAndVaud"
# KANTON = "Thurgau"

# PATH_TO_AOI = "projects/thurgau-irrigation/assets/FribourgAndVaud/broye_bounding_box"
# PATH_TO_AOI = f"projects/thurgau-irrigation/assets/{KANTON}/Bezirke_Contours_simplified_WGS84_Stammheim"
# PATH_TO_AOI = f"projects/thurgau-irrigation/assets/{KANTON}/Hoheitsgebiete_Contours_WGS84_simplified"
adm1_units = ee.FeatureCollection('projects/thurgau-irrigation/assets/GIS/Kantone_simplified100m')

#filter the cantons Zürich, Schaffhausen, and Thurgau from adm1_units using the 'NAME' property
adm1_units = adm1_units.filter(ee.Filter.inList('NAME', ['Zürich', 'Schaffhausen', 'Thurgau']))


TEMPORAL_RESOLUTION = "dekadal"

# # #Broye
# PATH_TO_DOUBLE_CROPPING_COLLECTION = f"projects/thurgau-irrigation/assets/{KANTON}/DoubleCropping/crop_vegetation_period_broye_{YEAR}"
# PATH_TO_JURISDICTION = (f"projects/thurgau-irrigation/assets/{KANTON}/elevation_bands_broye_v2_WGS84")
# PATH_TO_LANDUSE = (f"projects/thurgau-irrigation/assets/{KANTON}/filtered_permanent_pastures")
# LANDUSE_PROPERTY_NAME = "nutzung"

# Thurgau
PATH_TO_DOUBLE_CROPPING_COLLECTION = f"projects/thurgau-irrigation/assets/ZH_SH_TG/crop_vegetation_period_{YEAR}"
# PATH_TO_JURISDICTION = PATH_TO_AOI#f"projects/thurgau-irrigation/assets/{KANTON}/Bezirke_Contours_simplified_WGS84_Stammheim"
PATH_TO_LANDUSE_2024 = f"projects/thurgau-irrigation/assets/ZH_SH_TG/Nutzungsflaechen/ZH_SH_TG_2024_Kulturen"
PATH_TO_LANDUSE = f"projects/thurgau-irrigation/assets/ZH_SH_TG/Nutzungsflaechen/ZH_SH_TG_{YEAR}_Kulturen"
LANDUSE_PROPERTY_NAME = "nutzung"
# PATH_TO_LANDUSE_ZH = "projects/thurgau-irrigation/assets/ZH_Nutzungsflaechen_2/ZH_Nutzungsflaechen_2022"
# LANDUSE_PROPERTY_NAME_ZH = "NUTZUNG"

rainfed_reference_set = {
    "Extensiv genutzte Weiden",
    "Weiden (Heimweiden, übrige Weiden ohne Sömmerungsweiden)",
    "Übrige Dauerwiesen (ohne Weiden)",
    # "Übrige Grünfläche (Dauergrünfläche), beitragsberechtigt",
    # "Übrige Grünfläche (Dauergrünflächen), nicht beitragsberechtigt",
    "Extensiv genutzte Wiesen (ohne Weiden)",
    # "Waldweiden (ohne bewaldete Fläche)",
    
    # "Kunstwiesen (ohne Weiden)",
}

ET_GREEN_BAND_NAME = "ET_green"

# #Landsat
ET_BAND_NAME = "ET"
SCALING_FACTOR = 100
DYNAMIC = True
SCALING_FACTOR_PROPERTY_NAME = "days" if DYNAMIC else None
NUMBER_OF_IMAGES = 21
EXPORT_IMAGE_RESOLUTION = 30  # in meters
ET_BAND_RESOLUTION = 30  # 30# in meters

KANTON = "Thurgau"
PATH_TO_ET_PRODUCT_1 = f"projects/thurgau-irrigation/assets/{KANTON}/ET_products/decadal_Landsat_30m"
KANTON = "Zuerich"
PATH_TO_ET_PRODUCT_2 = f"projects/thurgau-irrigation/assets/{KANTON}/ET_products/decadal_Landsat_30m"
KANTON = "Schaffhausen"
PATH_TO_ET_PRODUCT_3 = f"projects/thurgau-irrigation/assets/{KANTON}/ET_products/decadal_Landsat_30m"

# ##Wapor
# ET_BAND_NAME ="downscaled"
# SCALING_FACTOR = 100  # scaling factor for the ET band to convert it back to float
# DYNAMIC = False #True #False, for wapor
# SCALING_FACTOR_PROPERTY_NAME = "days" if DYNAMIC else None
# ET_BAND_RESOLUTION = 10  # 30# in meters
# NUMBER_OF_IMAGES = 21
# EXPORT_IMAGE_RESOLUTION = 10  # in meters
# # PATH_TO_ET_PRODUCT = f"projects/thurgau-irrigation/assets/{KANTON}/ET_products/WaPOR_10m_{YEAR}"
# PATH_TO_ET_PRODUCT = f"projects/thurgau-irrigation/assets/ET_products/{KANTON}/WaPOR_10m_{YEAR}"


MINIMUM_FIELD_SIZE = 10000  # in square meters
MAXIMUM_NUMBER_OF_TREES = 5
ET_GREEN_ASSET_PATH = f"projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Pixels_{TEMPORAL_RESOLUTION}_from_Landsat_30m"
# ET_GREEN_ASSET_PATH = f"projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Weiden_Gemeinden_{TEMPORAL_RESOLUTION}_from_WaPOR_10m"

print(ET_GREEN_ASSET_PATH)


projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Pixels_dekadal_from_Landsat_30m


## 1. Load Assets

In [22]:
# aoi = ee.FeatureCollection(PATH_TO_AOI).geometry().simplify(500).buffer(100)
# aoi = ee.FeatureCollection(PATH_TO_AOI).geometry()
aoi = adm1_units.geometry()
jurisdictions = adm1_units

In [23]:
double_cropping_image = ee.Image(PATH_TO_DOUBLE_CROPPING_COLLECTION)
double_cropping_image = double_cropping_image.select('isDoubleCropping').where(double_cropping_image.select('secondStart').lte(5),0)

In [24]:
et_collection = (
    ee.ImageCollection(PATH_TO_ET_PRODUCT_1)
    .merge(ee.ImageCollection(PATH_TO_ET_PRODUCT_2))
    .merge(ee.ImageCollection(PATH_TO_ET_PRODUCT_3))
    .filterDate(f"{YEAR}-05-01", f"{YEAR}-09-30")
    .map(
        lambda img: back_to_float(
            img, SCALING_FACTOR, DYNAMIC, SCALING_FACTOR_PROPERTY_NAME
        )
    )
)

# Mosaic images with identical dates
et_collection = merge_same_date_images(et_collection).sort("system:time_start")

et_collection_list = et_collection.toList(et_collection.size())


# Sanity checks:
print_collection_dates(et_collection)
# print(f"Sizing of the ET collection: {et_collection.size().getInfo()}")

Dates of images in the collection:
2023-05-01
2023-05-11
2023-05-21
2023-06-01
2023-06-11
2023-06-21
2023-07-01
2023-07-11
2023-07-21
2023-08-01
2023-08-11
2023-08-21
2023-09-01
2023-09-11
2023-09-21


In [9]:
# image = ee.Image(et_collection_list.get(12))

# Map = geemap.Map()

# vis_params = {
#     "bands": [ET_BAND_NAME],
#     "min": 0,
#     "max": 3,
#     "palette": "viridis",
# }

# Map.addLayer(image, vis_params)
# Map.addLayer(aoi, {"color": "red"}, "AOI")
# Map.centerObject(aoi, 13)
# Map

In [27]:
landuse_collection = ee.FeatureCollection(PATH_TO_LANDUSE)


landuse_collection = (
    landuse_collection.map(lambda f: f.set("nutzung", f.get(LANDUSE_PROPERTY_NAME)))
    if LANDUSE_PROPERTY_NAME != "nutzung"
    else landuse_collection
)

print(
    f"Renamed {LANDUSE_PROPERTY_NAME} to 'nutzung'"
    if LANDUSE_PROPERTY_NAME != "nutzung"
    else "Collection has 'nutzung' property"
)

# landuse_collection=landuse_collection_zh.merge(landuse_collection)


n_trees= ee.FeatureCollection(PATH_TO_LANDUSE_2024).reduceToImage(properties=['anzahl_bae'], reducer=ee.Reducer.first())

Collection has 'nutzung' property


## 2. Compute ET_green

In [ ]:
# process_et_green(
#     et_collection_list=et_collection_list,
#     landuse_collection=landuse_collection,
#     jurisdictions=jurisdictions,
#     double_cropping_image=double_cropping_image,
#     year=YEAR,
#     aoi=aoi,
#     asset_path=ET_GREEN_ASSET_PATH,
#     et_band_name=ET_BAND_NAME,
#     time_step_type=TEMPORAL_RESOLUTION,
#     resolution=ET_BAND_RESOLUTION,
#     minimum_field_size=MINIMUM_FIELD_SIZE,
#     rainfed_crops = rainfed_reference_set,
# )

Exporting ET_green_dekadal_2018_04_D1 for 2018 to projects/thurgau-irrigation/assets/Thurgau/ET_green/ET_green_Weiden_Gemeinden_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2018_04_D1
Exporting ET_green_dekadal_2018_04_D2 for 2018 to projects/thurgau-irrigation/assets/Thurgau/ET_green/ET_green_Weiden_Gemeinden_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2018_04_D2
Exporting ET_green_dekadal_2018_04_D3 for 2018 to projects/thurgau-irrigation/assets/Thurgau/ET_green/ET_green_Weiden_Gemeinden_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2018_04_D3
Exporting ET_green_dekadal_2018_05_D1 for 2018 to projects/thurgau-irrigation/assets/Thurgau/ET_green/ET_green_Weiden_Gemeinden_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2018_05_D1
Exporting ET_green_dekadal_2018_05_D2 for 2018 to projects/thurgau-irrigation/assets/Thurgau/ET_green/ET_green_Weiden_Gemeinden_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2018_05_D2
Exporting ET_green_dekadal_2018_05_D3 for 2018 to projects/thurgau-irrigation/assets/Thurg

In [ ]:
# process_et_green_std(
#     et_collection_list=et_collection_list,
#     landuse_collection=landuse_collection,
#     jurisdictions=jurisdictions,
#     double_cropping_image=double_cropping_image,
#     year=YEAR,
#     aoi=aoi,
#     asset_path=ET_GREEN_STD_ASSET_PATH,
#     et_band_name=ET_BAND_NAME,
#     time_step_type=TEMPORAL_RESOLUTION,
#     resolution=ET_BAND_RESOLUTION,
#     minimum_field_size=MINIMUM_FIELD_SIZE,
# )

Exporting ET_green_dekadal_2019_04_D1 for 2019 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_std_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2019_04_D1
Exporting ET_green_dekadal_2019_04_D2 for 2019 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_std_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2019_04_D2
Exporting ET_green_dekadal_2019_04_D3 for 2019 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_std_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2019_04_D3
Exporting ET_green_dekadal_2019_05_D1 for 2019 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_std_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2019_05_D1
Exporting ET_green_dekadal_2019_05_D2 for 2019 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_std_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2019_05_D2
Exporting ET_green_dekadal_2019_05_D3 for 2019 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_gr

In [ ]:
# Process ET green for rainfed fields with 30m inward buffer
processed_et_collection = process_et_green_rainfed_buffered(
    et_collection_list=et_collection_list,
    landuse_collection=landuse_collection,
    jurisdictions=jurisdictions,
    n_trees=n_trees,
    double_cropping_image=double_cropping_image,#actually, we dont need this
    year=YEAR,
    aoi=aoi,
    asset_path=ET_GREEN_ASSET_PATH,
    et_band_name=ET_BAND_NAME,
    time_step_type=TEMPORAL_RESOLUTION,
    resolution=ET_BAND_RESOLUTION,
    minimum_field_size=MINIMUM_FIELD_SIZE,
    max_trees=MAXIMUM_NUMBER_OF_TREES
    buffer_distance=-30.0,  # 30m inward buffer
    rainfed_crops=rainfed_reference_set,
    export_band_name="ET_green_buffered"
)

Exporting ET_green_buffered_buffered_dekadal_2023_05_D1 for 2023 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Pixels_dekadal_from_Landsat_30m/ET_green_buffered_buffered_dekadal_2023_05_D1
Exporting ET_green_buffered_buffered_dekadal_2023_05_D2 for 2023 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Pixels_dekadal_from_Landsat_30m/ET_green_buffered_buffered_dekadal_2023_05_D2
Exporting ET_green_buffered_buffered_dekadal_2023_05_D2 for 2023 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Pixels_dekadal_from_Landsat_30m/ET_green_buffered_buffered_dekadal_2023_05_D2
Exporting ET_green_buffered_buffered_dekadal_2023_05_D3 for 2023 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Pixels_dekadal_from_Landsat_30m/ET_green_buffered_buffered_dekadal_2023_05_D3
Exporting ET_green_buffered_buffered_dekadal_2023_05_D3 for 2023 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Pixels_dekadal_from_Landsat_30m/ET_gre

In [ ]:
# Export the processed collection to Earth Engine assets
export_tasks = export_et_green_collection(
    et_collection=processed_et_collection,
    asset_path=f"{ET_GREEN_ASSET_PATH}_buffered",
    aoi=aoi,
    resolution=ET_BAND_RESOLUTION
)

print(f"Created {len(export_tasks)} export tasks for buffered rainfed fields")

# Optionally start the tasks (uncomment the following lines to actually start exports)
# for task in export_tasks:
#     task.start()
#     print(f"Started task: {task.config['description']}")

In [ ]:
# Visualize the buffered ET values
Map = geemap.Map()

# Get a sample image from the processed collection
sample_image = ee.Image(processed_et_collection.first())
print(f"Sample image date: {sample_image.date().format('YYYY-MM-dd').getInfo()}")

# Visualization parameters
vis_params_buffered = {
    "bands": ["ET_green_buffered"],
    "min": 0,
    "max": 3,
    "palette": "viridis",
}

# Compare with original ET for the same date
original_et = ee.Image(et_collection.first()).select(ET_BAND_NAME)
vis_params_original = {
    "bands": [ET_BAND_NAME],
    "min": 0,
    "max": 3,
    "palette": "plasma",
}

Map.center_object(aoi, 12)
Map.addLayer(sample_image, vis_params_buffered, "ET Buffered Rainfed Fields")
Map.addLayer(original_et, vis_params_original, "Original ET")
Map.addLayer(jurisdictions, {"color": "black"}, "Jurisdictions")
Map.add_colorbar(vis_params_buffered, label="ET [mm/day]", layer_name="ET Buffered Rainfed Fields")

Map

In [62]:
assetdir = f"projects/thurgau-irrigation/assets/{'FribourgAndVaud/ET_products/decadal_Landsat_30m'}"

assets = ee.data.listAssets({"parent": assetdir})["assets"]
for img in assets:
    id = img['id']
    asset = ee.data.getAsset(id)
    name = id.split('decadal_Landsat_30m/')[1]
    if (name.find('Thurgau') > 0):
        ee.data.copyAsset(id, 'projects/thurgau-irrigation/assets/Thurgau/ET_products/decadal_Landsat_30m/' + name)
        print('asset name: ' + name)
        ee.data.deleteAsset(id)

    # ee.data.copyAsset(id, 'projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_postprocessed/ET_blue_postprocessed_dekadal_from_Landsat_30m_v2/' + name)


asset name: ET_Landsat_decadal_Thurgau_201804_1
asset name: ET_Landsat_decadal_Thurgau_201804_2
asset name: ET_Landsat_decadal_Thurgau_201804_3
asset name: ET_Landsat_decadal_Thurgau_201805_1
asset name: ET_Landsat_decadal_Thurgau_201805_2
asset name: ET_Landsat_decadal_Thurgau_201805_3
asset name: ET_Landsat_decadal_Thurgau_201806_1
asset name: ET_Landsat_decadal_Thurgau_201806_2
asset name: ET_Landsat_decadal_Thurgau_201806_3
asset name: ET_Landsat_decadal_Thurgau_201807_1
asset name: ET_Landsat_decadal_Thurgau_201807_2
asset name: ET_Landsat_decadal_Thurgau_201807_3
asset name: ET_Landsat_decadal_Thurgau_201808_1
asset name: ET_Landsat_decadal_Thurgau_201808_2
asset name: ET_Landsat_decadal_Thurgau_201808_3
asset name: ET_Landsat_decadal_Thurgau_201809_1
asset name: ET_Landsat_decadal_Thurgau_201809_2
asset name: ET_Landsat_decadal_Thurgau_201809_3
asset name: ET_Landsat_decadal_Thurgau_201810_1
asset name: ET_Landsat_decadal_Thurgau_201810_2
asset name: ET_Landsat_decadal_Thurgau_2

In [50]:
et_green_list = []


not_irrigated_crops = get_crops_to_exclude()
rainfed_crops = get_rainfed_reference_crops()

# Prepare rainfed fields
rainfed_fields = prepare_rainfed_fields(
    landuse_collection, double_cropping_image, not_irrigated_crops, rainfed_crops, MINIMUM_FIELD_SIZE
)

for img in range(NUMBER_OF_IMAGES):
    et_img = ee.Image(et_collection_list.get(img)).select(ET_BAND_NAME)

    et_green_img = compute_et_green(
        et_img, rainfed_fields, jurisdictions, et_band_name=ET_BAND_NAME
    )
    et_green_list.append(et_green_img)

In [13]:
# rainfed_fields.first().getInfo()

# distinct_features = rainfed_fields.distinct("nutzung_normalized")

# # Get a list of just the 'nutzung' values
# nutzung_values = distinct_features.aggregate_array("nutzung_normalized").getInfo()

# nutzung_values

# landuse_collection.first().getInfo()

In [51]:
Map = geemap.Map()

image = ee.Image(et_green_list[14])
image_ET = ee.Image(et_collection_list.get(14))

print(image.date().format("YYYY-MM-dd").getInfo())

vis_params = {
    "bands": ["ET_green"],
    "min": 0,
    "max": 3,
    "palette": "viridis",
}

vis_params_ET = {
    "bands": [ET_BAND_NAME],
    "min": 0,
    "max": 3,
    "palette": "viridis",
}



Map.center_object(aoi, 12)
Map.addLayer(image, vis_params, "ET green 10m")
Map.add_colorbar(vis_params, label="ET green [mm/month]", layer_name="ET green 10m")
Map.addLayer(image_ET, vis_params_ET, "ET 10m")
Map.addLayer(rainfed_fields.filterBounds(jurisdictions), {"color": "red"}, "Rainfed fields")
Map.addLayer(jurisdictions, {"color": "black"}, "jurisdictions")




Map

2019-08-21


Map(center=[46.78675332740657, 6.874554339425382], controls=(WidgetControl(options=['position', 'transparent_b…

### Comparing WaPOR to Landsat

In [15]:
# wapor_collection = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_2019_dekadal_from_WaPOR_10m"
# ).map(lambda img: back_to_float(img, 100))

# landsat_collection = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_2019_dekadal_from_Landsat_30m"
# ).map(lambda img: back_to_float(img, 100))

In [16]:
# print_collection_dates(wapor_collection)
# print_collection_dates(landsat_collection)

In [17]:
# wapor_list = wapor_collection.toList(wapor_collection.size())
# landsat_list = landsat_collection.toList(landsat_collection.size())

In [18]:
# wapor_august = ee.Image(wapor_list.get(14))
# landsat_august = ee.Image(landsat_list.get(14))

# Map = geemap.Map()

# vis_params = {
#     "bands": ["ET_green"],
#     "min": 0,
#     "max": 3,
#     "palette": "viridis",
# }

# Map.center_object(aoi, 12)
# Map.addLayer(wapor_august, vis_params, "ET WaPOR")
# Map.addLayer(landsat_august, vis_params, "ET Landsat")
# Map.add_colorbar(vis_params, label="ET green [mm/month]")

# # Print the date of the images
# print(f"WaPOR image date: {ee.Image(wapor_august).date().format('YYYY-MM-dd').getInfo()}")

# Map

In [19]:
# landsat_august.date().format("YYYY-MM-dd").getInfo()